# Benders decomposition with decision optimization

This tutorial includes everything you need to set up decision optimization engines, build a mathematical programming model, then use the benders decomposition on it.


When you finish this tutorial, you'll have a foundational knowledge of _Prescriptive Analytics_.

>This notebook is part of **[Prescriptive Analytics for Python](http://ibmdecisionoptimization.github.io/docplex-doc/)**
>
>It requires either an [installation of CPLEX Optimizers](http://ibmdecisionoptimization.github.io/docplex-doc/getting_started.html) or it can be run on [IBM Watson Studio Cloud](https://www.ibm.com/cloud/watson-studio/) (Sign up for a [free IBM Cloud account](https://dataplatform.cloud.ibm.com/registration/stepone?context=wdp&apps=all>)
and you can start using Watson Studio Cloud right away).

Table of contents:

-  [Describe the business problem](#Describe-the-business-problem:--Games-Scheduling-in-the-National-Football-League)
*  [How decision optimization (prescriptive analytics) can help](#How--decision-optimization-can-help)
*  [Use decision optimization](#Use-decision-optimization)
    *  [Step 1: Import the library](#Step-1:-Import-the-library)
    *  [Step 2: Set up the prescriptive model](#Step-2:-Set-up-the-prescriptive-model)
    *  [Step 3: Solve the problem with default CPLEX algorithm](#Step-3:-Solve-the-problem-with-default-CPLEX-algorithm)
    *  [Step 4: Apply a Benders strategy](#Step-4:-Apply-a-Benders-strategy)
    *  [Step 5: Use the CPLEX annotations to guide CPLEX in your Benders decomposition](#Step-5:-Use-the-CPLEX-annotations-to-guide-CPLEX-in-your-Benders-decomposition)
*  [Summary](#Summary)
****

Benders decomposition is an approach to solve mathematical programming problems with a decomposable structure.

Starting with 12.7, CPLEX can decompose the model into a single master and (possibly multiple) subproblems. 

To do so, CPLEX can use of annotations that you supply for your model or can automatically do the decomposition. 
This approach can be applied to mixed-integer linear programs (MILP). For certain types of problems, this approach can offer significant performance improvements.

**Note**:
If your problem does not match such decomposition, CPLEX will raise an error at solve time.

CPLEX will produce an error CPXERR_BAD_DECOMPOSITION if the annotated decomposition does not yield disjoint subproblems

#### Learn more bout Benders decomposition

Directs a reader to more sources about Benders algorithm.

The popular acceptance of the original paper suggesting a decomposition or partitioning of a model to support solution of mixed integer programs gave rise to "Benders algorithm" as the name.

* *J. Benders*. <i>Partitioning procedures for solving mixed-variables programming problems in Numerische Mathematik, volume 4, issue 1, pages 238–252, 1962</i>

Other researchers developed the theory of cut-generating linear programs (CGLP) to further this practice.
* *M. Fischetti, D. Salvagnin, A. Zanette*. <i>A note on the selection of Benders’ cuts in Mathematical Programming, series B, volume 124, pages 175-182, 2010</i>

Still others applied the practice to practical operations research. This technical report describes Benders algorithm in "modern" terms and offers implementation hints.
* *M. Fischetti, I. Ljubic, M. Sinnl*. <i>Benders decomposition without separability: a computational study for capacitated facility location problems in Technical Report University of Padova, 2016</i>

## How  decision optimization can help

* Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes.  It takes into account specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control of business outcomes.  

* Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes.  

* Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle that future situation. Organizations that can act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.  
<br/>

<u>With prescriptive analytics, you can:</u> 

* Automate the complex decisions and trade-offs to better manage your limited resources.
* Take advantage of a future opportunity or mitigate a future risk.
* Proactively update recommendations based on changing events.
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.



## Use decision optimization

### Step 1: Import the library

Run the following code to import Decision Optimization CPLEX Modeling library.  The *DOcplex* library contains the two modeling packages, Mathematical Programming and Constraint Programming, referred to earlier.

In [31]:
import sys
try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')

A restart of the kernel might be needed.

### Step 2: Set up the prescriptive model

We will write a toy model just in order to show how to use the annotation api.

This model is not important: it just matche a benders decomposition, that is CPLEX can apply its new algorithm without any error.

The aim of this notebook is to discover and learn how to successfully apply Benders, not to see huge performance differences between a standard solve and a Benders based solve.

In [32]:
import numpy as np
# set seed to get consistent/comparable results
np.random.seed(1234) 

N = 20 # number of nodes in the graph
M = 10000 # a very large number

# Generate lower bound and upper bound
# The graph should be directed because for traffic simulation
# time from i to j could be vastly different than time from j to i
l = np.random.randint(low=1, high=10, size=(N,N))
u = np.random.randint(low=20, high=30, size=(N,N))

# Diaganol should be 0 as it represent a self loop for a node
for i in range(N):
    l[i][i] = 0
    u[i][i] = 0

# The graph may not be fully connected, thus, should delete some path
# Say the graph is 50% fully connected, set the weight of 50% of the
# path to a very large number M.
# The degree of connectivity in the graph is an adjustable parameter
dummy = np.random.randint(low=0, high=10, size=(N,N)) # matrix of 0 and 1
# For dummy = 1, keep path; otherwise, delete (set weight = M)
for i in range(N):
    for j in range(N):
        if dummy[i][j] != 0:
            l[i][j] = M
            u[i][j] = M    

# Arbituarily pick a source node and a distination node for testing
s = 0
t = 4

print('Lower bound matrix: \n', l)
print('\n')
print('Upper bound matrix: \n', u)

Lower bound matrix: 
 [[10000 10000 10000 10000 10000     2 10000 10000 10000 10000 10000 10000
  10000 10000     1 10000 10000     7 10000 10000]
 [10000 10000 10000 10000 10000 10000     4 10000 10000     8 10000 10000
  10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000     2 10000 10000 10000 10000 10000 10000
      5 10000 10000     7 10000 10000 10000 10000]
 [10000 10000     8 10000 10000 10000 10000 10000 10000 10000 10000     2
  10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000     4 10000 10000 10000 10000 10000
  10000 10000     9 10000 10000 10000 10000 10000]
 [    3 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000
  10000 10000 10000 10000 10000 10000     5 10000]
 [10000 10000 10000 10000     2 10000 10000     4 10000 10000 10000 10000
  10000 10000 10000 10000 10000 10000 10000 10000]
 [10000     7 10000 10000 10000 10000     2 10000 10000 10000 10000 10000
  10000 10000 10000 10000 100

Create one model instance, with a name. We set the log output to true such that we can see when CPLEX enables the Benders algorithm.

In [33]:
# first import the Model class from docplex.mp
from docplex.mp.model import Model

m = Model(name='benders', log_output=True)

# Binary variable: y_ij = 1 if arc(i,j) is on the shortest path
Y = m.binary_var_dict([(i,j) for i in range(N) for j in range(N)])

# Continuous variable: x_j = shortest path from s to j
# In fact, this would be integer for now because l and u
# are made of integers
X = m.continuous_var_dict(N)

In [34]:
# Objective function
m.minimize(m.sum(u[i][j]*Y[i,j] for i in range(N) for j in range(N)) - X[t])

# Constraints
m.add_constraint(X[s] == 0) # distance from s to s is 0
m.add_constraints(X[j] <= X[i]+l[i][j]+(u[i][j]-l[i][j])*Y[i,j] for i in range(N) for j in range(N))
m.add_constraint(m.sum(Y[s,k] for k in range(N)) - m.sum(Y[i,s] for i in range(N)) == 1) # source node flux = 1
m.add_constraint(m.sum(Y[t,k] for k in range(N)) - m.sum(Y[i,t] for i in range(N)) == -1) # source node flux = -1
for j in range(N): # intermediate node flux = 0
    if j != s and j != t:
        m.add_constraint(m.sum(Y[j,k] for k in range(N)) - m.sum(Y[i,j] for i in range(N)) == 0)

#### Solve with Decision Optimization 

If you're using a Community Edition of CPLEX runtimes, depending on the size of the problem, the solve stage may fail and will need a paying subscription or product installation.

You will get the best solution found after ***n*** seconds, thanks to a time limit parameter.

In [35]:
m.print_information()

Model: benders
 - number of variables: 420
   - binary=400, integer=0, continuous=20
 - number of constraints: 421
   - linear=421
 - parameters: defaults
 - problem type is: MILP


### Step 3: Solve the problem with default CPLEX algorithm

In [36]:
msol = m.solve()
assert msol is not None, "model can't solve"
m.report()

print(msol)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 10000.000000 after 0.00 sec. (0.02 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 366 rows and 25 columns.
MIP Presolve added 159 rows and 159 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 4 substitutions.
Reduced MIP has 210 rows, 550 columns, and 1334 nonzeros.
Reduced MIP has 380 binaries, 159 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.24 ticks)
Probing time = 0.00 sec. (0.14 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 159 rows and 159 columns.
MIP Presolve added 159 rows and 159 columns.
MIP Presolve modified 3 coefficients.
Reduced MIP has 210 rows, 550 columns, and 1334 nonzeros.
Reduced MIP has 380 binaries, 159 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.94 ticks)
Probing time = 0.00 sec. (0.14 ticks)
MIP emphasis: balance optimality and feasibi

#### Inspect the CPLEX Log.

If you inspect the CPLEX, you will see that it is a very standard log.
CPLEX needed 63 iterations to solve it.

In [37]:
obj1 = m.objective_value

### Step 4: Apply a Benders strategy

CPLEX implements a default Benders decomposition in certain situations.

If you want CPLEX to apply a Benders strategy as it solves your problem, but you do not specify cpxBendersPartition annotations yourself, CPLEX puts all integer variables in master and continuous variables into subproblems. 
If there are no integer variables in your model, or if there are no continuous variables in your model, CPLEX raises an error stating that it cannot automatically decompose the model to apply a Benders strategy.

You just need to set the Benders strategy parameter.

CPLEX supports 4 values for this parameter, from -1 to 3:
* OFF (default value) will ignore Benders.
* AUTO, USER, WORKERS, FULL will enable Benders.

Refer to the CPLEX documentation to understand the differences between the 4 values that trigger it.

In [38]:
m.parameters.benders.strategy = 3

In [39]:
m.print_information()

Model: benders
 - number of variables: 420
   - binary=400, integer=0, continuous=20
 - number of constraints: 421
   - linear=421
 - parameters:
     parameters.benders.strategy = 3
 - problem type is: MILP


We call cplex solve, but with the <i>clean_before_solve</i> flag because we want it to forget everything about previous solve and solution.

In [40]:
msol = m.solve(clean_before_solve=True)
assert msol is not None, "model can't solve"
m.report()

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_Benders_Strategy                        3
Found incumbent of value 10000.000000 after 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 365 rows and 25 columns.
MIP Presolve modified 4 coefficients.
Reduced MIP has 56 rows, 395 columns, and 866 nonzeros.
Reduced MIP has 380 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.53 ticks)
Tried aggregator 1 time.
MIP Presolve modified 3 coefficients.
Reduced MIP has 56 rows, 395 columns, and 866 nonzeros.
Reduced MIP has 380 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.38 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0       -0.0000          10000.0000    Benders: 1        7  10

#### Inspect the CPLEX Log.

Inspect the CPLEX log: you can now see that the log are different and you can see the message
<code>
Benders cuts applied:  3
</code>
which proves CPLEX applied successfully

You can see that CPLEX needed only 61 cumulative iterations, while it needed 63 previously.